In [13]:
import requests, bs4, selenium, time
from selenium import webdriver

In [15]:
urlfine = "https://ocw.mit.edu/courses/wgs-s10-history-of-women-in-science-and-engineering-fall-2017/"
url = 'https://ocw.mit.edu/search/?s=department_course_numbers.sort_coursenum'
pref="https://ocw.mit.edu"
#fineurl = 'https://ocw.mit.edu/courses/1-34-waste-containment-and-remediation-technology-spring-2004/'
PATH = './driver/chromedriver.exe'
driver = webdriver.Chrome(PATH)
driver.get(url)
time.sleep(5)
html = driver.page_source
ntot = int(bs4.BeautifulSoup(html, 'html.parser').find_all("span", {"class" : "results-total-number"})[0].text)
oldlinks=[]
f = open('res_dataset.csv', 'wt')
f.write("id;link;disciplines;keywords;title;description;type;language;difficulty;format;duration;public;min_age;max_age\n")
f.close()
f=open('res_dataset.csv', 'a')
while ntot>len(oldlinks)+2630:
    html = driver.page_source
    links = set([l for l in bs4.BeautifulSoup(html, 'html.parser').find_all("a", {"class" : "w-100"}) if not (l in oldlinks)])
    for s in links:
        #datacollector("https://ocw.mit.edu" + s.attrs["href"], f)
        print(pref+s.attrs["href"])
    driver.execute_script("window.scrollBy(0,500)")
    oldlinks=oldlinks + [l for l in links if not (l in oldlinks)]
driver.quit()
f.close()
print(len(oldlinks))

C:\Users\Lenovo\AppData\Local\Temp/ipykernel_18484/3865248611.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


https://ocw.mit.edu/courses/1-018j-ecology-i-the-earth-system-fall-2009/
https://ocw.mit.edu/courses/1-033-mechanics-of-material-systems-an-energy-approach-fall-2003/
https://ocw.mit.edu/courses/1-020-ecology-ii-engineering-for-sustainability-spring-2008/
https://ocw.mit.edu/courses/1-017-computing-and-data-analysis-for-environmental-applications-fall-2003/
https://ocw.mit.edu/courses/1-022-introduction-to-network-models-fall-2018/
https://ocw.mit.edu/courses/1-011-project-evaluation-spring-2011/
https://ocw.mit.edu/courses/1-040-project-management-spring-2004/
https://ocw.mit.edu/courses/1-00-introduction-to-computers-and-engineering-problem-solving-spring-2012/
https://ocw.mit.edu/courses/1-010-uncertainty-in-engineering-fall-2008/
https://ocw.mit.edu/courses/1-012-introduction-to-civil-engineering-design-spring-2002/
10
